**Workflow**:

---

1. Import Packages
2. STL Instantiation
3. STL Actualized Seasonalities
4. FB Instantiation
5. FB Actualized Seasonalities (For comparison)

In [3]:
#Import Packages
import pandas as pd
from prophet import Prophet
from google.colab import files
import xlrd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.seasonal import MSTL
from tbats import TBATS



In [ ]:
#STL Instantiation

#1. Import
df = pd.read_excel('/content/CA Recharge.xlsx')
df['ds'] = pd.to_datetime(df['ds'])

#2. Data Cleaning

df = df.set_index(['ds'])
df = df.asfreq('D')
df['y'] = pd.to_numeric(df['y'], errors='coerce')
df['y'] =df['y'].interpolate()
df['y'] = df['y'].ffill().bfill()
df= df.sort_index()
print(df['y'].isna().sum())

#3. Weekly Seasonality fitting

act_weekly_seasonality = STL(df['y'], period=7)
act_weekly_model = act_weekly_seasonality.fit()
act_weekly_excel = pd.DataFrame({'date': df.index, 'y':act_weekly_model.observed, 'seasonal':act_weekly_model.seasonal})
act_weekly_excel.to_excel("act_weekly.xlsx")


#4. Monthly Seasonality fitting
act_monthly_seasonality = STL(df['y'], period=30)
act_monthly_model = act_monthly_seasonality.fit()
act_monthly_excel = pd.DataFrame({'date': df.index, 'y':act_monthly_model.observed, 'seasonal':act_monthly_model.seasonal})
act_monthly_excel = act_monthly_excel[(act_monthly_excel['date'].dt.month == 12) & (act_monthly_excel['date'].dt.year == 2024)]
act_monthly_excel.to_excel("act_monthly.xlsx")




0


In [ ]:
#STL v Prophet Sanity Check

#1. STL Weekly Actualized Seasonality
weekly_decompose_df = act_weekly_model.seasonal.to_frame('seasonal')
weekly_decompose_df['weekday'] = weekly_decompose_df.index.day_name()
weekly_summary = weekly_decompose_df.groupby('weekday')['seasonal'].mean()
weekly_summary.to_excel('STL_weekly_summary.xlsx')

#2. STL Monthly Actualized Seasonality
monthly_decompose_df = act_monthly_model.seasonal.to_frame('seasonal')
monthly_decompose_df['day_of_month'] = monthly_decompose_df.index.day
monthly_summary = monthly_decompose_df.groupby('day_of_month')['seasonal'].mean()
monthly_summary.to_excel('STL_monthly_summary.xlsx')

In [8]:
# TBATS

#1. Import

df = pd.read_excel('/content/CA Recharge.xlsx')
df['ds'] = pd.to_datetime(df['ds'])

y = df['y'].values

model = TBATS(use_box_cox = True, use_trend=True,use_damped_trend=True,use_arma_errors=True, seasonal_periods=[7,30,365])

model_fit= model.fit(y)
print(model_fit.summary())

Streaming output truncated to the last 5000 lines.
    self.session.send(
  File "/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py", line 820, in send
    msg = self.msg(
          ^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py", line 661, in msg
    header = self.msg_header(msg_type) if header is None else header
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py", line 644, in msg_header
    return msg_header(self.msg_id, msg_type, self.username, self.session)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py", line 275, in msg_header
    date = utcnow()
           ^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py", line 203, in utcnow
    return datetime.utcnow().replace(tzinfo=utc)
           ^^^^^^^^^^^^^^^^^

During handling of the above

Use Box-Cox: True
Use trend: True
Use damped trend: True
Seasonal periods: [  7.  30. 365.]
Seasonal harmonics [3 1 1]
ARMA errors (p, q): (0, 0)
Box-Cox Lambda 1.000000
Smoothing (Alpha): 0.414414
Trend (Beta): 0.003854
Damping Parameter (Phi): 0.947784
Seasonal Parameters (Gamma): [-4.36822487e-08 -1.16024911e-07 -8.68756066e-08 -5.48416066e-08
  1.23242104e-07 -2.07919401e-07]
AR coefficients []
MA coefficients []
Seed vector [ 36201901.20233567   -914143.14826869   -478908.71857338
   -540070.75757726  -1444003.19375334    628974.23119223
    256230.51887734    -45406.64166256   -611148.75147845
    928688.51870209 -11279513.96822331   4318164.49140679]

AIC 38648.421242


In [ ]:
#Output file

raw_file = model_fit.seasonal_components_
seasonal_periods = model_fit.seasonal_periods
components = model_fit.components_per_period

seasonal_effects ={}

idx = np.cumsum([0]+list(components))

for i, p in enumerate(seasonal_periods):
    start, end = idx[i],idx[i+1]
    seasonal_effects[p] = raw_file[:,start:end].sum(axis=1)

name_map = { 7: 'weekly_effect', 30 : 'monthly_effect', 365: 'annual_effect'}

for p, values in seasonal_effects.items():
  df_new[name_map[p]] = values

df_new['tbats_trend']  = tbats_model.level
df_new['tbats_fitted'] = tbats_model.y_hat
df_new['tbats_resid']  = df-new['y'] - df_new['tbats_fitted']

# 5. OPTIONAL: Export
df_new.to_excel('TBATS_decomposition.xlsx', index=False)





In [ ]:
#Fourier Instantiation

#1. Import
df_ca = pd.read_excel('/content/CA Recharge.xlsx')
df_ca['ds'] = pd.to_datetime(df_ca['ds'])

#2. Model fitting
m1 = Prophet(daily_seasonality=False, yearly_seasonality=False, seasonality_mode='additive')
m1.add_seasonality(name='weekly', period=7, fourier_order=3)
m1.add_seasonality(name='monthly', period=30, fourier_order=5)
m1.add_seasonality(name='annual', period=365, fourier_order=10)
m1.add_country_holidays(country_name='PAK')
m1.fit(df_ca)

#2. Forecast
df_ca_future = m1.make_future_dataframe(periods=7)
forecast_ca = m1.predict(df_ca_future)
forecast_ca.to_excel('forecast_ca.xlsx')







INFO:prophet:Found custom seasonality named 'weekly', disabling built-in 'weekly' seasonality.


In [ ]:
#Fourier Actualized Seasonalities

#1. Weekly actualized Seasonality
weekly_decompose_df2 = forecast_ca[['ds','yhat','weekly']].copy()
weekly_decompose_df2['weekday'] = weekly_decompose_df2['ds'].dt.day_name()
weekly_summary = weekly_decompose_df2.groupby('weekday')['weekly'].mean()
weekly_summary.to_excel('FB_weekly_summary.xlsx')

#2. Monthly Actualized Seasonality
monthly_decompose_df2 = forecast_ca[['ds','yhat','monthly']].copy()
monthly_decompose_df2['day_of_month'] = monthly_decompose_df2['ds'].dt.day
#monthly_summary = monthly_decompose_df2.groupby('day_of_month')['monthly'].mean()
monthly_summary = monthly_decompose_df2[(monthly_decompose_df2['ds'].dt.month == 12) & (monthly_decompose_df2['ds'].dt.year == 2024)]
monthly_summary = monthly_summary.groupby('day_of_month')['monthly'].mean()
monthly_summary.to_excel('FB_monthly_summary.xlsx')

In [ ]:
from prophet.diagnostics import cross_validation, performance_metrics

df_cv = cross_validation(m, initial = '540 days', period = '30 days', horizon = '30 days')

df_p = performance_metrics(df_cv)
df_p.to_excel('df_p.xlsx')
df_cv.to_excel('df_cv.xlsx')

In [ ]:
from google.colab import drive

forecast.to_excel('/content/forecast.xlsx', index=False)

In [ ]:
!git clone https://github.com/Sannan27/glowing-adventure.git

%cd glowing-adventure

!git config user.name "abc"
!git config user.email "abc"

!git add .
!git commit -m"Recharge Forecast v 1.0"
!git push

In [ ]:
%cd /content
!rm -rf glowing-adventure

/content


In [ ]:
#MSTL
df = pd.read_excel('/content/Ufone.xlsx')
df['ds'] = pd.to_datetime(df['ds'])

#Canvas

df = df.set_index(['ds'])
df = df.asfreq('D')
df['y'] = pd.to_numeric(df['y'], errors='coerce')
df['y'] =df['y'].interpolate()
df['y'] = df['y'].ffill().bfill()
df= df.sort_index()
print(df['y'].isna().sum())

#Model instantiation

mstl = MSTL(df['y'], periods=[7], stl_kwargs = {'seasonal':7})
mstl_re = mstl.fit()
mstl_df = pd.DataFrame(mstl_re.seasonal, index = df.index, columns=['weekly'])

weekly_seasonality = mstl_df['weekly'].to_frame('seasonality_index')
weekly_seasonality['weekday'] = weekly_seasonality.index.day_name()
#weekly_seasonality = weekly_seasonality.dropna()

weekly_summary = weekly_seasonality.groupby('weekday')['seasonality_index'].mean()
print(weekly_seasonality.isna().sum())







0
seasonality_index    1023
weekday                 0
dtype: int64
